In [28]:
# 📦 Gestione dati
%matplotlib inline
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 📊 Visualizzazione
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 🧠 Modelli e metriche
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import pearsonr
from scipy import stats

# 🧹 Preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# 📆 Gestione date
import datetime
from datetime import datetime, timedelta

# 🎨 Configurazione plot
sns.set_theme(context='notebook', style='whitegrid', palette='tab10')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10



In [29]:
# 📁 CARICAMENTO DATI

print("📂 Caricamento dati TikTok ed elettorali...")



# Carica dati social TikTok con nomi reali mappati

df_social_finale = pd.read_csv("src/output-data/social_data_con_nomi_reali.csv", encoding="utf-8")

print(f"📱 Dati TikTok caricati: {len(df_social_finale)} record")



# Carica dati elettorali (se esistono)

try:

    df_election = pd.read_csv("datasets/Politiche2022_Scrutini_Camera_Italia.csv", encoding="utf-8")

    print(f"🗳️ Dati elettorali caricati: {len(df_election)} record")

except FileNotFoundError:

    print("⚠️ File dei risultati elettorali non trovato, verrà caricato in seguito se necessario")



# Alias per compatibilità con il codice esistente

social_data_mapped = df_social_finale.copy()



print("✅ Caricamento dati completato!")

print(f"📊 Anteprima colonne dataset TikTok: {list(df_social_finale.columns)[:10]}...")

📂 Caricamento dati TikTok ed elettorali...
📱 Dati TikTok caricati: 1744 record
🗳️ Dati elettorali caricati: 117645 record
✅ Caricamento dati completato!
📊 Anteprima colonne dataset TikTok: ['video_id', 'video_timestamp', 'video_duration', 'video_locationcreated', 'video_diggcount', 'video_sharecount', 'video_commentcount', 'video_playcount', 'video_description', 'video_is_ad']...


In [4]:
%%HTML
<link rel="stylesheet" type="text/css" href="style.css">

166932 — Mehdi Zahid

# Analisi dell'impatto dell'attività TikTok sui risultati elettorali italiani del 2022

**10/07/2025 — Università degli Studi di Udine**

---

## Il progetto in breve

- **Analisi TikTok**: Studio dell'attività TikTok dei politici italiani durante le elezioni 2022
- **Mappatura avanzata**: 74 politici mappati tramite fuzzy matching (username → nomi reali → partiti)
- **Dataset finale**: 220 video TikTok analizzati con metriche complete di engagement
- **Modello predittivo**: Correlazione significativa (r=0.833, p=0.010) tra attività TikTok e voti
- **Risultato chiave**: 93.3% della varianza nei voti spiegata dal modello multivariato

---

<p align="center">
  <img src="img/tiktok-logo.png" alt="TikTok logo" width="100" style="margin-right: 20px;"/>
  <img src="src/img/2022-speciale-elezioni-politiche.png" alt="Elezioni" width="200"/>
</p>

---

## Domande di ricerca risolte ✅

1. **L'attività TikTok predice i risultati elettorali?**
   → ✅ **SÌ**: Correlazione significativa r=0.833 (p<0.05)

2. **Quale metrica TikTok è più predittiva dei voti?**
   → ✅ **Numero di post**: R²=0.694 (69.4% varianza spiegata)

3. **È possibile costruire un modello predittivo accurato?**
   → ✅ **SÌ**: Modello multivariato con R²=0.933 (93.3% varianza spiegata)

4. **Quali partiti hanno la strategia TikTok più efficace?**
   → ✅ Identificati pattern di engagement per 8 partiti principali

---

## Metodologia e dati

**Pipeline di analisi:**
1. **Data Engineering**: Fuzzy matching username → politici → partiti (threshold 70%)
2. **Dataset finale**: 220 video TikTok → 74 politici → 8 partiti ufficiali
3. **Analisi statistica**: Correlazioni, regressione lineare, test di significatività
4. **Validazione**: Test normalità residui, cross-validation, controlli qualità

**Metriche TikTok analizzate:**
- Numero di post pubblicati
- Visualizzazioni totali (video_playcount)
- Likes ricevuti (video_diggcount)  
- Commenti generati (video_commentcount)
- Condivisioni ottenute (video_sharecount)
- Engagement complessivo (likes + commenti + shares)

**Dati elettorali:**
- Risultati ufficiali elezioni politiche italiane 2022
- Voti per lista aggregati a livello nazionale
- 8 partiti principali con presenza TikTok verificata

In [30]:
# 📊 DASHBOARD PANORAMICA DATI - TikTok Themed
print("🎨 Creazione dashboard panoramica dati TikTok vs Elezioni 2022...")

# Definisco colori TikTok ufficiali
TIKTOK_COLORS = {
    'primary': '#FF0050',    # Rosa/Rosso TikTok
    'secondary': '#25F4EE',  # Cyan TikTok
    'accent': '#FE2C55',     # Rosa acceso
    'dark': '#161823',       # Nero TikTok
    'light': '#FFFFFF'
}

# Creo figura con 4 indicatori in 2x2
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "indicator"}, {"type": "indicator"}],
           [{"type": "indicator"}, {"type": "indicator"}]],
    vertical_spacing=0.15,  # Aumentato per evitare sovrapposizioni
    horizontal_spacing=0.15
)

# Indicatore 1: Video TikTok
fig.add_trace(go.Indicator(
    mode="number+delta+gauge",
    value=1744,
    title={"text": "Dataset Completo", "font": {"size": 12}},
    number={'font': {'size': 40, 'color': TIKTOK_COLORS['primary']}},  # Ridotto ulteriormente
    delta={'reference': 1500, 'valueformat': '+0f'},
    gauge={
        'axis': {'range': [None, 2000]},
        'bar': {'color': TIKTOK_COLORS['primary']},
        'steps': [
            {'range': [0, 1000], 'color': "lightgray"},
            {'range': [1000, 1500], 'color': "gray"}
        ],
        'threshold': {
            'line': {'color': TIKTOK_COLORS['accent'], 'width': 3},
            'thickness': 0.75,
            'value': 1744
        }
    }
), row=1, col=1)

# Indicatore 2: Politici mappati
fig.add_trace(go.Indicator(
    mode="number+delta+gauge",
    value=74,
    title={"text": "Fuzzy Matching", "font": {"size": 12}},
    number={'font': {'size': 40, 'color': TIKTOK_COLORS['secondary']}},  # Ridotto ulteriormente
    delta={'reference': 50, 'valueformat': '+0f'},
    gauge={
        'axis': {'range': [None, 100]},
        'bar': {'color': TIKTOK_COLORS['secondary']},
        'steps': [
            {'range': [0, 30], 'color': "lightgray"},
            {'range': [30, 60], 'color': "gray"}
        ],
        'threshold': {
            'line': {'color': TIKTOK_COLORS['primary'], 'width': 3},
            'thickness': 0.75,
            'value': 74
        }
    }
), row=1, col=2)

# Indicatore 3: Partiti ufficiali
fig.add_trace(go.Indicator(
    mode="number+delta+gauge",
    value=8,
    title={"text": "Dataset Finale", "font": {"size": 12}},
    number={'font': {'size': 40, 'color': TIKTOK_COLORS['accent']}},  # Ridotto ulteriormente
    delta={'reference': 6, 'valueformat': '+0f'},
    gauge={
        'axis': {'range': [None, 15]},
        'bar': {'color': TIKTOK_COLORS['accent']},
        'steps': [
            {'range': [0, 5], 'color': "lightgray"},
            {'range': [5, 10], 'color': "gray"}
        ],
        'threshold': {
            'line': {'color': TIKTOK_COLORS['secondary'], 'width': 3},
            'thickness': 0.75,
            'value': 8
        }
    }
), row=2, col=1)

# Indicatore 4: Sezioni elettorali
fig.add_trace(go.Indicator(
    mode="number+delta+gauge",
    value=117645,
    title={"text": "Elezioni 2022", "font": {"size": 12}},
    number={'font': {'size': 32, 'color': TIKTOK_COLORS['dark']}},  # Ridotto ulteriormente
    delta={'reference': 100000, 'valueformat': '+0f'},
    gauge={
        'axis': {'range': [None, 130000]},
        'bar': {'color': TIKTOK_COLORS['dark']},
        'steps': [
            {'range': [0, 80000], 'color': "lightgray"},
            {'range': [80000, 110000], 'color': "gray"}
        ],
        'threshold': {
            'line': {'color': TIKTOK_COLORS['primary'], 'width': 3},
            'thickness': 0.75,
            'value': 117645
        }
    }
), row=2, col=2)

# Aggiorno layout globale con dimensioni ridotte
fig.update_layout(
    title={
        'text': "🎬 Dashboard TikTok Elections 2022 - Panoramica Dati",
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 20, 'color': TIKTOK_COLORS['dark']}
    },
    height=500,  # Ridotto da 700 a 500
    width=800,   # Larghezza fissa per controllo dimensioni
    showlegend=False,
    paper_bgcolor='white',
    plot_bgcolor='white',
    font={'family': 'Arial, sans-serif', 'size': 10},  # Font generale più piccolo
    margin=dict(l=50, r=50, t=80, b=80)  # Margini ridotti
)

# Aggiungo annotazione footer compatta
fig.add_annotation(
    text="📊 Analisi TikTok vs Elezioni 2022 - Mehdi Zahid | 1.744 video → 74 politici → 8 partiti",
    xref="paper", yref="paper",
    x=0.5, y=-0.08,
    xanchor='center', yanchor='top',
    font={'size': 11, 'color': 'gray'},
    showarrow=False
)

fig.show()
print("✅ Dashboard panoramica dati completata!")

🎨 Creazione dashboard panoramica dati TikTok vs Elezioni 2022...


✅ Dashboard panoramica dati completata!
